In [3]:
import os
import wyscoutapi
import pandas as pd

def load_environment_vars():
    # Get the current working directory
    cwd = os.getcwd()

    # Append the .env file name
    env_path = os.path.join(cwd, '.env')

    # Open the .env file
    with open(env_path) as f:
        # Read the lines
        lines = f.readlines()

    # Parse the key-value pairs
    for line in lines:
        # Ignore comments and empty lines
        if line.startswith('#') or not line.strip():
            continue
        # Split the line into key and value
        key, value = line.strip().split('=', 1)
        # Set the environment variable
        os.environ[key] = value

    return os.environ['WYSCOUT_API_USER'], os.environ['WYSCOUT_API_PASSWORD']

def create_wyscout_client(username, password):
    return wyscoutapi.WyscoutAPI(username=username, password=password)

username, password = load_environment_vars()

#username = "a4cp2s4-scgbfrd0o-vawk6my-fwwi4kbqyp"
#password = "8Ns;^pxxdMx+Ox!L5Dr_$MI*+c11RR"
client = create_wyscout_client(username, password)

In [4]:
areas = client.areas()['areas']
areas_df = pd.DataFrame(areas)

In [6]:
areas_df[areas_df['name'] == 'Argentina']
eur_area = areas_df[areas_df['name'] == "Europe"]['alpha3code'].values[0]

competitions = client.competitions('ESP')['competitions']
competition_df = pd.DataFrame(competitions)

In [61]:
competition_df.query('gender == "male" and category == "default" and format == "Domestic league" and divisionLevel == 1')

,wyId,name,area,format,type,category,gender,divisionLevel
10,795,LaLiga,"{'id': 724, 'alpha2code': 'ES', 'alpha3code': ...",Domestic league,club,default,male,1


In [62]:
competitions = client.competitions(eur_area)['competitions']
competition_df = pd.DataFrame(competitions)

In [ ]:
# we select the male in the major category and club type
competition_df.query("gender == 'male' and category == 'default' and format == 'International cup' and type == 'club'")

In [8]:
competition_players = client.competition_players(competition_id=795)

In [ ]:
client.competition_players(competition_id=795)

In [28]:
laliga_teams = client.competition_teams(competition_id=795)['teams']

In [109]:
import json

# Define the file path for the JSON file
# Get the current working directory
cwd = os.getcwd()

# Specify the full absolute path of the file
file_path = os.path.join(cwd, 'laliga_teams.json')

# Export the laliga_teams variable to a JSON file
with open(file_path, 'w') as f:
    json.dump(laliga_teams, f)

In [77]:
teams_players = []
for id in laliga_teams:
    team_id = id['wyId']
    print(team_id)
    players = client.team_squad(team_id=team_id)
    teams_players += players['squad']

674
675
676
678
679
680
681
682
684
686
687
692
696
698
700
708
713
714
756
846


In [87]:
teams_players[0]

{'wyId': 4892,
 'shortName': 'José Gayá',
 'firstName': 'José Luis',
 'middleName': '',
 'lastName': 'Gayá Peña',
 'height': 172,
 'weight': 66,
 'birthDate': '1995-05-25',
 'birthArea': {'id': 724,
  'alpha2code': 'ES',
  'alpha3code': 'ESP',
  'name': 'Spain'},
 'passportArea': {'id': 724,
  'alpha2code': 'ES',
  'alpha3code': 'ESP',
  'name': 'Spain'},
 'role': {'name': 'Defender', 'code2': 'DF', 'code3': 'DEF'},
 'foot': 'left',
 'currentTeamId': 674,
 'currentNationalTeamId': None,
 'gender': 'male',
 'status': 'active',
 'imageDataURL': 'https://cdn5.wyscout.com/photos/players/public/g228139_100x130.png'}

In [90]:

player_stats_list = []
league_id = 795

for player in teams_players:
    player_stats = client.player_advancedstats(player['wyId'], league_id)['average']
    player_stats['shortName'] = player['shortName']
    player_stats['height'] = player['height']
    player_stats['weight'] = player['weight']
    player_stats['role'] = player['role']['code3']
    player_stats['foot'] = player['foot']
    player_stats['currentTeamId'] = player['currentTeamId']
    player_stats['league_id'] = league_id
    player_stats['player_id'] = player['wyId']
    
    player_stats_list.append(player_stats)
    
    print(player['wyId'])

4892
25638
171643
215265
282444
292654
352787
382622
396526
423157
446483
473364
475709
545811
584914
584938
584944
601297
621020
622390
631906
634666
712646
747916
769463
3304
3327
3397
3429
4498
4501
8287
14723
14724
14748
276979
326523
364005
403448
404397
406071
422517
480145
493295
547260
599551
617894
626223
703740
859506
3347
3671
7899
8032
14808
14817
15008
135103
135903
367389
373410
397303
408307
410062
470747
471753
618294
626130
631404
647161
770398
770402
770833
773942
776387
784394
3413
3418
3421
3425
3503
4452
5542
230857
230862
247141
282449
286963
339791
344059
344086
525104
545942
549129
552245
593442
593446
654823
712509
778363
855222
938660
118
3324
3443
3682
4375
4488
8311
25553
28922
41363
70123
70273
213948
227894
254408
279533
279538
294415
414193
471991
495674
513850
563651
776656
776659
776663
783174
312
3306
3474
5167
11063
20441
55095
86338
89073
140046
208942
247207
261127
282790
291528
299246
364845
397033
435759
447821
558739
583159
584761
589805
624613
6

In [111]:
import json

# Define the file path for the JSON file
# Get the current working directory
cwd = os.getcwd()

# Specify the full absolute path of the file
file_path = os.path.join(cwd, 'player_stats.json')

# Save the player_stats_list variable to a JSON file
with open(file_path, 'w') as f:
    json.dump(player_stats_list, f)

In [114]:
player_stats_list[0]

{'passLength': 21.06,
 'longPassLength': 29.5,
 'dribbleDistanceFromOpponentGoal': 24.16,
 'ballRecoveries': 7.78,
 'duels': 14.07,
 'defensiveDuels': 7.65,
 'offensiveDuels': 3.17,
 'aerialDuels': 1.31,
 'fouls': 0.95,
 'goals': 0.05,
 'assists': 0.09,
 'passes': 38.37,
 'smartPasses': 0.14,
 'passesToFinalThird': 4.21,
 'crosses': 3.76,
 'dribbles': 1.27,
 'shots': 0.41,
 'headShots': 0,
 'interceptions': 3.57,
 'successfulDefensiveAction': 9.05,
 'yellowCards': 0.09,
 'redCards': 0,
 'directRedCards': 0,
 'successfulAttackingActions': 1.95,
 'freeKicks': 0.18,
 'directFreeKicks': 0.05,
 'corners': 0,
 'penalties': 0,
 'accelerations': 0.59,
 'looseBallDuels': 1.95,
 'missedBalls': 0.14,
 'forwardPasses': 13.44,
 'backPasses': 6.47,
 'throughPasses': 0.41,
 'keyPasses': 0.5,
 'verticalPasses': 13.76,
 'longPasses': 5.29,
 'shotAssists': 0.9,
 'shotOnTargetAssists': 0.5,
 'linkupPlays': 0,
 'opponentHalfRecoveries': 1.86,
 'dangerousOpponentHalfRecoveries': 0.27,
 'ballLosses': 0.05,


In [96]:
laliga_df = pd.DataFrame(player_stats_list)

In [103]:
# Get the current working directory
cwd = os.getcwd()

# Specify the full absolute path of the file
file_path = os.path.join(cwd, 'train.csv')

laliga_df[['height', 'weight', 'role', 'foot', 'currentTeamId', 'league_id', 'goals']].to_csv(file_path)

In [104]:
laliga_df[['height', 'weight', 'role', 'foot', 'currentTeamId', 'league_id', 'goals']]

,height,weight,role,foot,currentTeamId,league_id,goals
0,172,66,DEF,left,674,795,0.05
1,183,78,DEF,right,674,795,0.00
2,191,83,FWD,right,674,795,0.26
3,185,73,GKP,right,674,795,0.00
4,177,75,FWD,right,674,795,0.07
...,...,...,...,...,...,...,...
527,191,87,DEF,right,846,795,0.00
528,175,70,DEF,right,846,795,0.11
529,190,76,MID,left,846,795,0.38
530,175,65,MID,right,846,795,0.14


In [ ]:
# Get the current working directory
cwd = os.getcwd()

# Specify the full absolute path of the file
file_path = os.path.join(cwd, 'laliga_players.csv')

# Save the dataframe to the specified file path
laliga_df.to_csv(file_path)

In [66]:
player_stats = client.player_advancedstats(4892, 795)['average']

In [70]:
player_stats

{'passLength': 21.06,
 'longPassLength': 29.5,
 'dribbleDistanceFromOpponentGoal': 24.16,
 'ballRecoveries': 7.78,
 'duels': 14.07,
 'defensiveDuels': 7.65,
 'offensiveDuels': 3.17,
 'aerialDuels': 1.31,
 'fouls': 0.95,
 'goals': 0.05,
 'assists': 0.09,
 'passes': 38.37,
 'smartPasses': 0.14,
 'passesToFinalThird': 4.21,
 'crosses': 3.76,
 'dribbles': 1.27,
 'shots': 0.41,
 'headShots': 0,
 'interceptions': 3.57,
 'successfulDefensiveAction': 9.05,
 'yellowCards': 0.09,
 'redCards': 0,
 'directRedCards': 0,
 'successfulAttackingActions': 1.95,
 'freeKicks': 0.18,
 'directFreeKicks': 0.05,
 'corners': 0,
 'penalties': 0,
 'accelerations': 0.59,
 'looseBallDuels': 1.95,
 'missedBalls': 0.14,
 'forwardPasses': 13.44,
 'backPasses': 6.47,
 'throughPasses': 0.41,
 'keyPasses': 0.5,
 'verticalPasses': 13.76,
 'longPasses': 5.29,
 'shotAssists': 0.9,
 'shotOnTargetAssists': 0.5,
 'linkupPlays': 0,
 'opponentHalfRecoveries': 1.86,
 'dangerousOpponentHalfRecoveries': 0.27,
 'ballLosses': 0.05,


In [94]:
pd.DataFrame(client.player_career(1022194)['career'])

,playerId,teamId,seasonId,competitionId,shirtNumber,goal,penalties,appearances,yellowCard,redCards,substituteIn,substituteOut,substituteOnBench,minutesPlayed
0,1022194,32280,189213,1723,13,0,0,0,0,0,0,0,3,0
1,1022194,8508,188921,108,43,0,0,0,0,0,0,0,5,0
2,1022194,8672,189271,1442,43,0,0,2,1,0,0,0,0,180


In [90]:
pd.DataFrame(client.competition_players(competition_id=108)['players'])

,wyId,shortName,firstName,middleName,lastName,height,weight,birthDate,birthArea,passportArea,role,foot,currentTeamId,currentNationalTeamId,gender,status,imageDataURL
0,1022194,J. Morrison,Joe,,Morrison,0,0,2004-06-01,"{'id': 1002, 'alpha2code': 'SC', 'alpha3code':...","{'id': 1002, 'alpha2code': 'SC', 'alpha3code':...","{'name': 'Goalkeeper', 'code2': 'GK', 'code3':...",None,8508,NaN,male,active,https://cdn5.wyscout.com/photos/players/public...
1,947407,D. Cardoso,Dylan,,de Jesus Cardoso,0,0,2004-06-16,"{'id': 442, 'alpha2code': 'LU', 'alpha3code': ...","{'id': 442, 'alpha2code': 'LU', 'alpha3code': ...","{'name': 'Defender', 'code2': 'DF', 'code3': '...",left,14637,14769.0,male,active,https://cdn5.wyscout.com/photos/players/public...
2,945296,A. Idrizi,Ardi,,Idrizi,0,0,2004-03-28,"{'id': 807, 'alpha2code': 'MK', 'alpha3code': ...","{'id': 807, 'alpha2code': 'MK', 'alpha3code': ...","{'name': 'Defender', 'code2': 'DF', 'code3': '...",None,60413,NaN,male,active,https://cdn5.wyscout.com/photos/players/public...
3,945257,S. Bolaji,Samson,,Bolaji Ajayi,180,75,1999-03-03,"{'id': 566, 'alpha2code': 'NG', 'alpha3code': ...","{'id': 566, 'alpha2code': 'NG', 'alpha3code': ...","{'name': 'Midfielder', 'code2': 'MD', 'code3':...",right,19812,NaN,male,active,https://cdn5.wyscout.com/photos/players/public...
4,945249,R. Thompson,Reece,,Thompson,0,0,2004-10-08,"{'id': 826, 'alpha2code': 'EN', 'alpha3code': ...","{'id': 826, 'alpha2code': 'EN', 'alpha3code': ...","{'name': 'Goalkeeper', 'code2': 'GK', 'code3':...",None,1968,NaN,male,active,https://cdn5.wyscout.com/photos/players/public...
5,944828,A. Keko,Albers,,Keko,185,75,2001-08-16,"{'id': 8, 'alpha2code': 'AL', 'alpha3code': 'A...","{'id': 8, 'alpha2code': 'AL', 'alpha3code': 'A...","{'name': 'Forward', 'code2': 'FW', 'code3': 'F...",right,8675,NaN,male,active,https://cdn5.wyscout.com/photos/players/public...
6,943602,I. Brusberg,Isak,,Brusberg,188,78,2006-09-05,"{'id': 752, 'alpha2code': 'SE', 'alpha3code': ...","{'id': 752, 'alpha2code': 'SE', 'alpha3code': ...","{'name': 'Midfielder', 'code2': 'MD', 'code3':...",right,6736,31758.0,male,active,https://cdn5.wyscout.com/photos/players/public...
7,943588,A. Arnarsson,Arnar Smári,,Arnarsson,0,0,2004-05-04,"{'id': 352, 'alpha2code': 'IS', 'alpha3code': ...","{'id': 352, 'alpha2code': 'IS', 'alpha3code': ...","{'name': 'Forward', 'code2': 'FW', 'code3': 'F...",None,7732,NaN,male,active,https://cdn5.wyscout.com/photos/players/public...
8,943586,Á. Orrason,Ásgeir Helgi,,Orrason,0,0,2005-05-19,"{'id': 352, 'alpha2code': 'IS', 'alpha3code': ...","{'id': 352, 'alpha2code': 'IS', 'alpha3code': ...","{'name': 'Defender', 'code2': 'DF', 'code3': '...",right,7725,7843.0,male,active,https://cdn5.wyscout.com/photos/players/public...
9,943584,Ilde Serrano,Ildefons Josep,,Serrano Valiño,0,0,1989-01-13,"{'id': 20, 'alpha2code': 'AD', 'alpha3code': '...","{'id': 20, 'alpha2code': 'AD', 'alpha3code': '...","{'name': 'Goalkeeper', 'code2': 'GK', 'code3':...",None,14900,NaN,male,active,https://cdn5.wyscout.com/photos/players/public...


In [69]:
pd.DataFrame(matches['matches'])

,matchId,label,date,dateutc,status,competitionId,seasonId,roundId,gameweek
0,5467186,"Borussia Dortmund - Real Madrid, 0-2",2024-06-01 21:00:00,2024-06-01 19:00:00,Played,108,188921,4429424,1
1,5467184,"Real Madrid - Bayern München, 2-1",2024-05-08 21:00:00,2024-05-08 19:00:00,Played,108,188921,4429423,2
2,5467185,"PSG - Borussia Dortmund, 0-1",2024-05-07 21:00:00,2024-05-07 19:00:00,Played,108,188921,4429423,2
3,5467183,"Borussia Dortmund - PSG, 1-0",2024-05-01 21:00:00,2024-05-01 19:00:00,Played,108,188921,4429423,1
4,5467182,"Bayern München - Real Madrid, 2-2",2024-04-30 21:00:00,2024-04-30 19:00:00,Played,108,188921,4429423,1
...,...,...,...,...,...,...,...,...,...
423,5344860,"Malmö FF - Víkingur Reykjavík, 3-2",2022-07-05 19:00:00,2022-07-05 17:00:00,Played,108,188032,4426693,1
424,5344846,"Pyunik - CFR Cluj, 0-0",2022-07-05 18:00:00,2022-07-05 16:00:00,Played,108,188032,4426693,1
425,5344845,"Inter Club d'Escaldes - Víkingur Reykjavík, 0-1",2022-06-24 21:30:00,2022-06-24 19:30:00,Played,108,188032,4426692,1
426,5344843,"Levadia - Víkingur Reykjavík, 1-6",2022-06-21 21:30:00,2022-06-21 19:30:00,Played,108,188032,4426691,1


In [71]:
players = client.competition_players(competition_id=108)

In [78]:
pd.DataFrame(players['players'])

,wyId,shortName,firstName,middleName,lastName,height,weight,birthDate,birthArea,passportArea,role,foot,currentTeamId,currentNationalTeamId,gender,status,imageDataURL
0,1022194,J. Morrison,Joe,,Morrison,0,0,2004-06-01,"{'id': 1002, 'alpha2code': 'SC', 'alpha3code':...","{'id': 1002, 'alpha2code': 'SC', 'alpha3code':...","{'name': 'Goalkeeper', 'code2': 'GK', 'code3':...",None,8508,NaN,male,active,https://cdn5.wyscout.com/photos/players/public...
1,947407,D. Cardoso,Dylan,,de Jesus Cardoso,0,0,2004-06-16,"{'id': 442, 'alpha2code': 'LU', 'alpha3code': ...","{'id': 442, 'alpha2code': 'LU', 'alpha3code': ...","{'name': 'Defender', 'code2': 'DF', 'code3': '...",left,14637,14769.0,male,active,https://cdn5.wyscout.com/photos/players/public...
2,945296,A. Idrizi,Ardi,,Idrizi,0,0,2004-03-28,"{'id': 807, 'alpha2code': 'MK', 'alpha3code': ...","{'id': 807, 'alpha2code': 'MK', 'alpha3code': ...","{'name': 'Defender', 'code2': 'DF', 'code3': '...",None,60413,NaN,male,active,https://cdn5.wyscout.com/photos/players/public...
3,945257,S. Bolaji,Samson,,Bolaji Ajayi,180,75,1999-03-03,"{'id': 566, 'alpha2code': 'NG', 'alpha3code': ...","{'id': 566, 'alpha2code': 'NG', 'alpha3code': ...","{'name': 'Midfielder', 'code2': 'MD', 'code3':...",right,19812,NaN,male,active,https://cdn5.wyscout.com/photos/players/public...
4,945249,R. Thompson,Reece,,Thompson,0,0,2004-10-08,"{'id': 826, 'alpha2code': 'EN', 'alpha3code': ...","{'id': 826, 'alpha2code': 'EN', 'alpha3code': ...","{'name': 'Goalkeeper', 'code2': 'GK', 'code3':...",None,1968,NaN,male,active,https://cdn5.wyscout.com/photos/players/public...
5,944828,A. Keko,Albers,,Keko,185,75,2001-08-16,"{'id': 8, 'alpha2code': 'AL', 'alpha3code': 'A...","{'id': 8, 'alpha2code': 'AL', 'alpha3code': 'A...","{'name': 'Forward', 'code2': 'FW', 'code3': 'F...",right,8675,NaN,male,active,https://cdn5.wyscout.com/photos/players/public...
6,943602,I. Brusberg,Isak,,Brusberg,188,78,2006-09-05,"{'id': 752, 'alpha2code': 'SE', 'alpha3code': ...","{'id': 752, 'alpha2code': 'SE', 'alpha3code': ...","{'name': 'Midfielder', 'code2': 'MD', 'code3':...",right,6736,31758.0,male,active,https://cdn5.wyscout.com/photos/players/public...
7,943588,A. Arnarsson,Arnar Smári,,Arnarsson,0,0,2004-05-04,"{'id': 352, 'alpha2code': 'IS', 'alpha3code': ...","{'id': 352, 'alpha2code': 'IS', 'alpha3code': ...","{'name': 'Forward', 'code2': 'FW', 'code3': 'F...",None,7732,NaN,male,active,https://cdn5.wyscout.com/photos/players/public...
8,943586,Á. Orrason,Ásgeir Helgi,,Orrason,0,0,2005-05-19,"{'id': 352, 'alpha2code': 'IS', 'alpha3code': ...","{'id': 352, 'alpha2code': 'IS', 'alpha3code': ...","{'name': 'Defender', 'code2': 'DF', 'code3': '...",right,7725,7843.0,male,active,https://cdn5.wyscout.com/photos/players/public...
9,943584,Ilde Serrano,Ildefons Josep,,Serrano Valiño,0,0,1989-01-13,"{'id': 20, 'alpha2code': 'AD', 'alpha3code': '...","{'id': 20, 'alpha2code': 'AD', 'alpha3code': '...","{'name': 'Goalkeeper', 'code2': 'GK', 'code3':...",None,14900,NaN,male,active,https://cdn5.wyscout.com/photos/players/public...
